In [1]:
import logging

In [2]:
logger = logging.getLogger("mylogger")
logger.setLevel(logging.INFO)

def add(a, b):
    try:
        result = a + c
    except Exception as e:
        print(e)

In [3]:
c = add(10, 'Bob')

name 'c' is not defined


In [4]:
import pandas as pd
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
SOURCE_PATH = os.path.dirname(os.getcwd())

N_JOBS = 1  # number of jobs to run in parallel
N_SPLITS = 10  # number of splits (k) to be made within the k fold cv
N_ITER = 50  # number of parameter settings that are sampled
SEED = 2294
VERBOSE = True
INPUTS_PATH = os.path.join(SOURCE_PATH, "data", "inputs")
OUTPUTS_PATH = os.path.join(SOURCE_PATH, "data", "outputs")
DATASET_NAMES = ["betadgp_covdgp_data", "betadgp_beta2x2_data", "betadgp_data"]
TARGET_NAME = "betas_dgp"
MODEL_TAG = "ffnn"
STANDARDIZE = True
TRAIN_SIZE = 0.7 # size of training data (in %)

VALIDATION_SIZE = 0.2

In [7]:
DIR_NAMES = os.listdir(os.path.join(INPUTS_PATH))

inputs_path = INPUTS_PATH
target_name = TARGET_NAME
standardize = STANDARDIZE
train_size = TRAIN_SIZE

dir_name = DIR_NAMES[0]
d_name = DATASET_NAMES[0] 

train_data = pd.read_csv(os.path.join(inputs_path, dir_name, d_name + ".csv"))
train_data.set_index(["Var1", "Var2"], inplace=True)
y_train = train_data[[target_name]].to_numpy()
X_train = train_data.drop([target_name], axis=1).to_numpy()

test_data = pd.read_csv(os.path.join(inputs_path, dir_name, d_name + "_test.csv"))
test_data.set_index(["Var1", "Var2"], inplace=True)
y_test = test_data[[target_name]].to_numpy()
X_test = test_data.drop([target_name], axis=1).to_numpy()

if standardize:
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=train_size)

    scaler = StandardScaler()
    X_train_zscore = scaler.fit_transform(X_train)
    X_validation_zscore = scaler.transform(X_validation)
    X_test_zscore = scaler.transform(X_test)

In [10]:
from scipy.stats import reciprocal
import numpy as np

def FFNN(n_hidden,
         n_neurons,
         input_shape,
         learning_rate,
         activation):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))

    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation))
    model.add(keras.layers.Dense(1))

    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss=loss, optimizer=optimizer)

    return model

class FFNNWrapper():
    def __init__(self, model_params=None):
        self.model_name = "ffnn"
        self.search_type = 'random'
        self.param_grid = {"n_hidden": [1, 2, 3, 4, 5],
                           "n_neurons": np.arange(1, 100+1),
                           "learning_rate": reciprocal(3e-4, 3e-2),
                           "activation": ["relu"],
                           "loss": ["mse"]}
        self.ModelClass = keras.wrappers.scikit_learn.KerasRegressor(FFNN)

        if model_params is not None:
            self.param_grid.update(model_params)
            
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import RandomizedSearchCV, KFold

cv_splits = KFold(n_splits=N_SPLITS)
mse_scorer = make_scorer(mean_squared_error)

wrapper = FFNNWrapper(model_params={"input_shape": [X_train_zscore.shape[1]]})

model_search = RandomizedSearchCV(estimator=wrapper.ModelClass,
                                  param_distributions=wrapper.param_grid,
                                  n_iter=N_ITER,
                                  cv=cv_splits,
                                  verbose=0,
                                  n_jobs=N_JOBS,
                                  scoring=mse_scorer,
                                  random_state=SEED) 

In [14]:
import traceback

In [17]:
try:
    opt_model_search = model_search.fit(X_train_zscore,
                                    y_train,
                                    epochs=50,
                                    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
                                    validation_data=(X_validation_zscore, y_validation))
except:
    var = traceback.format_exc()
    
    # open file
    log_file = open(os.path.join(d_name + ".log"), "w")

    # write error
    log_file.write(str(var))

    # close file
    log_file.close()

In [13]:
opt_model_search = model_search.fit(X_train_zscore,
                                    y_train,
                                    epochs=50,
                                    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
                                    validation_data=(X_validation_zscore, y_validation))

ValueError: loss is not a legal parameter